# Develop a CasADi Model of The Steady-State Solar Plant RTO Problem

Replicates the calculations in the following Excel spreadsheet and uses CasADi to solve the optimization problem.

- `SS Solar Plant Optimization New Thermal & Flow Model 15 DVs I-O 2025-10-12a.xlsm`

In [ ]:
import os
import casadi as cas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm import tqdm

# Import functions from the solar_plant_rto module
from problems.solar_plant_rto.solar_plant_gen_rto import (
    PUMP_SPEED_MIN,
    PUMP_SPEED_MAX,
    COLLECTOR_D_INT,
    actual_pump_speed_from_scaled,
    calculate_pump_and_drive_efficiency,
    calculate_pump_dp,
    calculate_pump_fluid_power,
    calculate_collector_flow_rate,
    calculate_collector_oil_exit_temp,
    solar_plant_rto_solve,
    steam_generator_solve,
    solar_plant_gen_rto_solve
)

In [ ]:
plot_dir = "plots"
os.makedirs(plot_dir, exist_ok=True)

## Test Data

In [ ]:
from tests.test_solar_plant_gen_rto import test_data, test_params

# This data is from the Excel spreadsheet
test_data

## Plant Configuration Parameters

In [ ]:
m_pumps = test_params['m_pumps']
n_lines = test_params['n_lines']

## Expected Range of Oil Flow Rates

In [ ]:
# Operating point temperature is 330 degC
oil_rho = 636.52  # kg/m^3

# Range of oil flow rates to consider
# This is based on the high operating point from Luis of 6.25 kg/s
collector_oil_m_dot = np.linspace(1.5, 7.0, 12)

# Volumetric flow rates (m^3/h)
collector_oil_flow_rate = collector_oil_m_dot * 3600 / oil_rho

# Equivalent range of oil flow velocities (m/s)
oil_flow_velocity_values = (
    collector_oil_flow_rate / (3600 * np.pi * COLLECTOR_D_INT ** 2 / 4)
)

total_oil_m_dot = n_lines * collector_oil_m_dot
total_oil_flow_rate = n_lines * collector_oil_flow_rate

collector_oil_flow_range = pd.DataFrame({
    "Flow rate (kg/s)": collector_oil_m_dot,
    "Flow rate (m^3/h)": collector_oil_flow_rate,
    "Flow velocity (m/s)": oil_flow_velocity_values
})
total_oil_flow_range = pd.DataFrame({
    "Flow rate (kg/s)": total_oil_m_dot,
    "Flow rate (m^3/h)": total_oil_flow_rate
})

pd.concat({
    "Collector Oil Flows": collector_oil_flow_range,
    f"Total Flow ({n_lines} Lines)": total_oil_flow_range
}, axis=1).round(2)

## Pump Characteristics

In [ ]:
flow_rates = np.linspace(50, 120, 71)

fig, axes = plt.subplots(3, 1, sharex=True, figsize=(7, 5.5))

for pump_speed in [1500, 2000, 2500, 3000]:
    efficiency = calculate_pump_and_drive_efficiency(
        flow_rates, pump_speed
    )
    delta_p = calculate_pump_dp(
        pump_speed, flow_rates, m_pumps
    )
    power = calculate_pump_fluid_power(flow_rates, delta_p)

    ax = axes[0]
    ax.plot(flow_rates, delta_p, label=pump_speed)
    ax.set_ylabel('dP (kPa)')
    ax.grid(True)
    ax.legend(title="Speed (rpm)")
    ax.set_title("Pump Curves")


    ax = axes[1]
    ax.plot(flow_rates, efficiency * 100, label=pump_speed)
    ax.set_ylabel('Efficiency (%)')
    ax.grid(True)
    ax.legend(title="Speed (rpm)")
    ax.set_title("Efficiency Curves")

    ax = axes[2]
    ax.plot(flow_rates, power, label=pump_speed)
    ax.set_xlabel('Flow rate (m³/h)')
    ax.set_ylabel('Power (kW)')
    ax.grid(True)
    ax.legend(title="Speed (rpm)")
    ax.set_title("Power Curves")

plt.tight_layout()
plt.show()

## Collector Line Flow Characteristics

In [ ]:
valve_positions = np.linspace(0.2, 1.0, 41)

fig, ax = plt.subplots(figsize=(7, 2.5))

for delta_p in [1.5, 2.0, 2.5]:
    flow_rates = calculate_collector_flow_rate(
        valve_positions, delta_p, sqrt=np.sqrt
    )

    ax.plot(valve_positions, flow_rates, label=delta_p)
    ax.set_xlabel('Valve position (0-1)')
    ax.set_ylabel('Flow rate (m³/h)')
    ax.grid(True)
    ax.legend(title="Pressure drop (kPa)")
    ax.set_title("Collector Loop Flow Curves")

plt.tight_layout()
plt.show()

## Oil Exit Temperature Calculations

## Heat Exchanger Heat Transfer Coefficient Calculation

In [ ]:
from problems.solar_plant_rto.solar_plant_gen_rto import (
    calculate_actual_heat_transfer_coefficient,
    HX1_U_LIQUID,
    HX2_U_BOIL,
    HX3_U_STEAM,
    F_OIL_NOMINAL
)

# Same range as collector loop exit temperature plots
flow_rates = np.linspace(80, 120, 41)  # Total oil flow rate (m^3/h)

# Calculate actual heat transfer coefficients
U_steam_actual = calculate_actual_heat_transfer_coefficient(
    flow_rates, HX3_U_STEAM, F_oil_nominal=F_OIL_NOMINAL
)
U_boil_actual = calculate_actual_heat_transfer_coefficient(
    flow_rates, HX2_U_BOIL, F_oil_nominal=F_OIL_NOMINAL
)
U_liquid_actual = calculate_actual_heat_transfer_coefficient(
    flow_rates, HX1_U_LIQUID, F_oil_nominal=F_OIL_NOMINAL
)

# Create the plot
fig, ax = plt.subplots(figsize=(7, 3.5))

ax.plot(flow_rates, U_steam_actual, label='U_steam (HX3)', marker='o', markersize=3)
ax.plot(flow_rates, U_boil_actual, label='U_boil (HX2)', marker='s', markersize=3)
ax.plot(flow_rates, U_liquid_actual, label='U_liquid (HX1)', marker='^', markersize=3)

ax.set_xlabel('Total Oil Flow Rate ($m^3/h$)')
ax.set_ylabel('Heat Transfer Coefficient ($W/m^2K$)')
ax.set_title('Heat Exchanger Heat Transfer Coefficients vs. Oil Flow Rate')
ax.grid(True)
ax.legend()

plt.tight_layout()
plt.show()

## Power Generator Output

In [ ]:
pd.Series({
    name: test_data[name] for name in
    ['total_flow_rate', 'mixed_oil_exit_temp', 'pump_and_drive_power', 'steam_power', 'net_power']
})

In [ ]:
# Maximizes net power generation

oil_flow_rates = np.linspace(70.0, 90.0, 21)  # kg/s
mixed_oil_exit_temps = [385, 390, 395, 400]  # degC

# Silence solver output
solver_opts = {'ipopt.tol': 1e-9, 'print_time': False, 'ipopt': {'print_level': 0}}

results = {
    "m_dot": defaultdict(list),
    "T1": defaultdict(list),
    "T2": defaultdict(list),
    "Tr": defaultdict(list),
    "steam_power": defaultdict(list),
    "hx_area_error": defaultdict(list)
}

for mixed_oil_exit_temp in tqdm(mixed_oil_exit_temps):

    for oil_flow_rate in tqdm(oil_flow_rates):

        sol, variables = steam_generator_solve(
            mixed_oil_exit_temp,
            oil_flow_rate,
            m_dot_init=0.75,
            T_steam_sp=mixed_oil_exit_temp - 10.0,  # Adjust with oil temp
            solver_opts=solver_opts
        )

        for name in ['m_dot', 'T1', 'T2', 'Tr', 'steam_power', 'hx_area_error']:
            results[name][mixed_oil_exit_temp].append(variables[name])

results = pd.concat({name: pd.DataFrame(data) for name, data in results.items()}, axis=1)
results.columns.names = ['Variable', 'Mixed Oil Exit Temp']
results.index = pd.Index(oil_flow_rates, name='Oil Flow Rate (m^3/h)')
results.round(3)

In [ ]:
print(results['Tr'].round(2))

In [ ]:
plot_info = {
    'm_dot': {'name': 'Steam Flow Rate', 'units': 'kg/s'},
    'T1': {'name': 'Oil Temperature after HX1', 'units': 'deg C'},
    'T2': {'name': 'Oil Temperature after HX2', 'units': 'deg C'},
    'Tr': {'name': 'Oil Return Temperature', 'units': 'deg C'},
    'steam_power': {'name': 'Steam Produced', 'units': 'kW'},
 }

n_plots = len(plot_info)

fig, axes = plt.subplots(n_plots, 1, sharex=True, figsize=(7, 1 + 1.5 * n_plots))

for ax, (name, info) in zip(axes, plot_info.items()):
    for mixed_oil_exit_temp in results[name].columns:
        results[name][mixed_oil_exit_temp].plot(
            ax=ax,
            style='.-',
            title=f"{info['name']} vs. Oil Flow Rate and Temperature",
            xlabel='Oil Flow Rate ($m^3/h$)',
            ylabel=info['units'],
            label=f'{mixed_oil_exit_temp} °C'
        )
    ylim = ax.get_ylim()
    if ylim[1] - ylim[0] < 1.0:
        ax.set_ylim([np.floor(ylim[0]), np.ceil(ylim[1])])
    ax.grid(True)
    ax.legend(title='Oil Temp')

plt.tight_layout()
plt.savefig(os.path.join(plot_dir, "steam_gen_solve_results.png"), dpi=150)
plt.show()

## Solar Collector Lines and Pumps RTO Model

In [ ]:
# Maximizes net potential energy generation

ambient_temp = 20  # degC
solar_rates = np.linspace(450, 800, 36)  # W/m2
oil_return_temp = 270  # degC

# Silence solver output
solver_opts = {'ipopt.tol': 1e-9, 'print_time': False, 'ipopt': {'print_level': 0}}

valve_positions = []
oil_flow_rates = []
oil_exit_temps = []
pump_speeds = []
potential_work = []
index_values = []

for solar_rate in tqdm(solar_rates):

    sol, variables = solar_plant_rto_solve(
        solar_rate,
        ambient_temp,
        oil_return_temp,
        m_pumps,
        n_lines,
        solver_opts=solver_opts,
    )
    actual_pump_speed = actual_pump_speed_from_scaled(
        variables['pump_speed_scaled']
    )

    valve_positions.append(variables['valve_positions'])
    oil_flow_rates.append(variables['collector_flow_rates'])
    oil_exit_temps.append(variables['oil_exit_temps'])
    pump_speeds.append(actual_pump_speed)
    potential_work.append(variables['potential_work'])
    index_values.append(solar_rate)

valve_positions = pd.DataFrame.from_records(
    valve_positions,
    index=pd.Index(index_values, name='Solar Rate (W/m2)'),
    columns=pd.Index(range(n_lines), name='Collector Line')
)
oil_flow_rates = pd.DataFrame.from_records(
    oil_flow_rates,
    index=pd.Index(index_values, name='Solar Rate (W/m2)'),
    columns=pd.Index(range(n_lines), name='Collector Line')
)
oil_exit_temps = pd.DataFrame.from_records(
    oil_exit_temps,
    index=pd.Index(index_values, name='Solar Rate (W/m2)'),
    columns=pd.Index(range(n_lines), name='Collector Line')
)
pump_speeds = pd.Series(
    pump_speeds,
    index=pd.Index(index_values, name='Pump Speed (RPM)'),
    name='Pump Speed (scaled)'
)
potential_work = pd.Series(
    potential_work,
    index=pd.Index(index_values, name='Potential Useful Work Output (kW)'),
    name='Pump Speed (scaled)'
)


In [ ]:
fig, axes = plt.subplots(5, 1, sharex=True, figsize=(7, 8.5))

n_lines_to_plot = min(8, n_lines)

ax = axes[0]
for line in oil_exit_temps.columns[:n_lines_to_plot]:  # Plot first 8 lines only
    oil_exit_temps[line].plot(
        ax=ax,
        style='.-',
        title=f'Collector Lines 1 to {n_lines_to_plot} Exit Temperatures',
        xlabel='Solar Rate (W/m2)',
        ylabel='Oil Temperature (°C)',
    )
ax.set_ylim([390, 401])
ax.grid(True)

ax = axes[1]
for line in valve_positions.columns[:n_lines_to_plot]:  # Plot first 8 lines only
    valve_positions[line].plot(
        ax=ax,
        style='.-',
        title=f'Collector Lines 1 to {n_lines_to_plot} FCV Positions',
        xlabel='Solar Rate (W/m2)',
        ylabel='Valve Position (0-1)',
    )
ax.set_ylim([None, 1.01])
ax.grid(True)

ax = axes[2]
for line in oil_flow_rates.columns[:n_lines_to_plot]:  # Plot first 8 lines only
    oil_flow_rates[line].plot(
        ax=ax,
        style='.-',
        title=f'Collector Lines 1 to {n_lines_to_plot} Oil Flow Rates',
        xlabel='Solar Rate ($W/m^2$)',
        ylabel='Flow rate ($m^3/h$)',
    )
ax.grid(True)

ax = axes[3]
pump_speeds.plot(
    ax=ax,
    style='.-',
    title=f'Oil Pump Speed',
    xlabel='Solar Rate ($W/m^2$)',
    ylabel='Pump Speed (RPM)',
    label='Actual speed',
)
ax.axhline(PUMP_SPEED_MIN, color='red', linestyle='--', label='Min speed')
ax.axhline(PUMP_SPEED_MAX, color='red', linestyle='--', label='Max speed')
ax.grid(True)
ax.legend()

ax = axes[4]
potential_work.plot(
    ax=ax,
    style='.-',
    title=f'Potential Useful Work Output',
    xlabel='Solar Rate ($W/m^2$)',
    ylabel='Work (kW)',
)
ax.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(plot_dir, "solar_plant_rto_results.png"), dpi=150)
plt.show()

## Combined Solar Collector Lines, Pumps and Power Generator RTO Model

In [ ]:
# Maximizes net power generation

ambient_temp = 20  # degC
solar_rates = np.linspace(450, 800, 36)  # W/m2

# Silence solver output
solver_opts = {'ipopt.tol': 1e-9, 'print_time': False, 'ipopt': {'print_level': 0}}

valve_positions = []
oil_flow_rates = []
oil_exit_temps = []
pump_speeds = []
net_power_output = []
index_values = []

for solar_rate in tqdm(solar_rates):

    sol, variables = solar_plant_gen_rto_solve(
        ambient_temp,
        solar_rate,
        n_lines,
        m_pumps,
        valve_positions_init=0.9,
        pump_speed_scaled_init=0.3,
        m_dot_init=0.75,
        oil_return_temp_init=260.0,
        oil_rho_cp=636.52 * 2.138,
        solver_opts=solver_opts
    )
    actual_pump_speed = actual_pump_speed_from_scaled(
        variables['pump_speed_scaled']
    )

    valve_positions.append(variables['valve_positions'])
    oil_flow_rates.append(variables['collector_flow_rates'])
    oil_exit_temps.append(variables['oil_exit_temps'])
    pump_speeds.append(actual_pump_speed)
    net_power_output.append(variables['net_power'])
    index_values.append(solar_rate)

valve_positions = pd.DataFrame.from_records(
    valve_positions,
    index=pd.Index(index_values, name='Solar Rate (W/m2)'),
    columns=pd.Index(range(n_lines), name='Collector Line')
)
oil_flow_rates = pd.DataFrame.from_records(
    oil_flow_rates,
    index=pd.Index(index_values, name='Solar Rate (W/m2)'),
    columns=pd.Index(range(n_lines), name='Collector Line')
)
oil_exit_temps = pd.DataFrame.from_records(
    oil_exit_temps,
    index=pd.Index(index_values, name='Solar Rate (W/m2)'),
    columns=pd.Index(range(n_lines), name='Collector Line')
)
pump_speeds = pd.Series(
    pump_speeds,
    index=pd.Index(index_values, name='Solar Rate (W/m2)'),
    name='Pump Speed (RPM)'
)
net_power_output = pd.Series(
    net_power_output,
    index=pd.Index(index_values, name='Solar Rate (W/m2)'),
    name='Net Power Output (kW)'
)

In [ ]:
fig, axes = plt.subplots(5, 1, sharex=True, figsize=(7, 8.5))

n_lines_to_plot = min(8, n_lines)

ax = axes[0]
for line in oil_exit_temps.columns[:n_lines_to_plot]:  # Plot first 8 lines only
    oil_exit_temps[line].plot(
        ax=ax,
        style='.-',
        title=f'Collector Lines 1 to {n_lines_to_plot} Oil Exit Temperatures',
        xlabel='Solar Rate ($W/m^2$)',
        ylabel='Oil Temperature (°C)',
    )
ax.set_ylim([390, 401])
ax.grid(True)

ax = axes[1]
for line in valve_positions.columns[:n_lines_to_plot]:  # Plot first 8 lines only
    valve_positions[line].plot(
        ax=ax,
        style='.-',
        title=f'Collector Lines 1 to {n_lines_to_plot} FCV Positions',
        xlabel='Solar Rate ($W/m^2$)',
        ylabel='Valve Position (0-1)',
    )
ax.set_ylim([None, 1.01])
ax.grid(True)

ax = axes[2]
for line in oil_flow_rates.columns[:n_lines_to_plot]:  # Plot first 8 lines only
    oil_flow_rates[line].plot(
        ax=ax,
        style='.-',
        title=f'Collector Lines 1 to {n_lines_to_plot} Flow Rates',
        xlabel='Solar Rate ($W/m^2$)',
        ylabel='Flow rate ($m^3/h$)',
    )
ax.grid(True)

ax = axes[3]
pump_speeds.plot(
    ax=ax,
    style='.-',
    title=f'Oil Pump Speed',
    xlabel='Solar Rate ($W/m^2$)',
    ylabel='Pump Speed (RPM)',
    label='Actual speed',
)
ax.axhline(PUMP_SPEED_MIN, color='red', linestyle='--', label='Min speed')
ax.axhline(PUMP_SPEED_MAX, color='red', linestyle='--', label='Max speed')
ax.grid(True)
ax.legend()

ax = axes[4]
net_power_output.plot(
    ax=ax,
    style='.-',
    title=f'Net Power Generated',
    xlabel='Solar Rate ($W/m^2$)',
    ylabel='Power (kW)',
)
ax.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(plot_dir, "solar_plant_rto_gen_results.png"), dpi=150)
plt.show()